In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import quantstats as qs
import warnings
import os
os.chdir('/Users/fulinq/Documents/KMITL/FinancialEngineering/Y4/Y4T1/PROJECT/ARDL-ECM/Code/Thai/quantstat')

# 1. Invalid frequency: ME
if not hasattr(pd.tseries.offsets, 'ME'):
    pd.tseries.offsets.ME = pd.tseries.offsets.MonthEnd

# 2. Resampler.sum axis error
from pandas.core.resample import Resampler
_original_resampler_sum = Resampler.sum
def _patched_resampler_sum(self, *args, **kwargs):
    if 'axis' in kwargs: kwargs.pop('axis')
    return _original_resampler_sum(self, *args, **kwargs)
Resampler.sum = _patched_resampler_sum


def create_rl_portfolio_report(file_name, stock_name, output_file='RL_LSTM_Portfolio_Report.html', shift=0):
    
    print(f"🚀 Processing: {file_name} -> {stock_name}")
    
    # 1. Load Data (Assumes Excel based on your snippet)
    df = pd.read_excel(file_name)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    # 2. Fetch Market Data
    start_date = df.index.min()
    end_date = df.index.max()
    
    # Download Data
    price_data = yf.download(stock_name, start=start_date, end=end_date)
    
    # Fix MultiIndex (Directly droplevel as requested)
    if isinstance(price_data.columns, pd.MultiIndex):
        price_data.columns = price_data.columns.droplevel(1)

    # 3. Calculate Returns (Log Returns)
    price_data['Returns'] = np.log(price_data['Close']).diff()
    
    # 4. Align Signal
    df['Aligned_Signal'] = df['Signal'].shift(shift)

    # 5. Merge Data
    portfolio = pd.merge(price_data['Returns'], df['Aligned_Signal'], left_index=True, right_index=True, how='inner')
    
    # Strategy Returns
    portfolio['Strategy_Returns'] = portfolio['Aligned_Signal'] * portfolio['Returns']
    portfolio.dropna(inplace=True)

    # 6. Generate Report
    print(f"📊 Generating Report: {output_file}")
    qs.extend_pandas()
    qs.reports.html(
        portfolio['Strategy_Returns'], 
        benchmark=portfolio['Returns'], 
        output=output_file,
        title=f'RL-LSTM Strategy vs {stock_name}'
    )
    
    print(f"✅ DONE! File saved: {output_file}")


file_name = 'DL_Predictions_V33_Metrics_KKP.xlsx'
output_file = 'KKP.html'
stock_name = 'KKP.BK'
create_rl_portfolio_report(file_name, stock_name, output_file)

🚀 Processing: DL_Predictions_V33_Metrics_KKP.xlsx -> KKP.BK


/var/folders/rc/w7r40l35143bxwkzfxdygctc0000gn/T/ipykernel_35895/733678384.py:36: FutureWarning: YF.download() has changed argument auto_adjust default to True
  price_data = yf.download(stock_name, start=start_date, end=end_date)
/Users/fulinq/.pyenv/versions/3.12.5/lib/python3.12/site-packages/yfinance/scrapers/history.py:204: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  dt_now = pd.Timestamp.utcnow()
[*********************100%***********************]  1 of 1 completed


📊 Generating Report: KKP.html
✅ DONE! File saved: KKP.html


/Users/fulinq/.pyenv/versions/3.12.5/lib/python3.12/site-packages/quantstats/_plotting/core.py:1016: ChainedAssignmentError: A value is being set on a copy of a DataFrame or Series through chained assignment using an inplace method.
Such inplace method never works to update the original DataFrame or Series, because the intermediate object on which we are setting values always behaves as a copy (due to Copy-on-Write).

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' instead, to perform the operation inplace on the original object, or try to avoid an inplace operation using 'df[col] = df[col].method(value)'.

See the documentation for a more detailed explanation: https://pandas.pydata.org/pandas-docs/stable/user_guide/copy_on_write.html
  port["Weekly"].ffill(inplace=True)
/Users/fulinq/.pyenv/versions/3.12.5/lib/python3.12/site-packages/quantstats/_plotting/core.py:1019: ChainedAssignmentError: A value is being set on a co